<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/intro-Machine-Learning/blob/main/classes/class_18/class_18_pipeline_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

# Clase 18 

Pasos para creación del cuaderno interactivo.

1. Plantear bien la pregunta.  

   * ¿Regresión o clasificación?
   * ¿Tipo de regresión y tipo de clasificación?

2. Exploración inicial.
   * Indicar la fuente de dónde se toman los datos.
   * Hacer explícita la función objetivo.
   * Decir cuáles son los atributos (descripción breve de cada uno)
   * Practicar una primera s´ntesis tabular y una exploración gráfica de los datos.
   
3. Preparar los datos para los algoritmos de aprendizaje.

   * Explorar correlaciones lineales con la variable objetivo.
   * Eliminar de ser necesario atributos que no sean de mucha utilidad.
   * Hacer separación inicial de datos para entrenar y para testear.
   * Limpiar datos y llenar datos faltantes.
   * Estandarizar los datos.
   * Crear funciones en Python de manera que se puedan replicar los procesos de transformación de datos en proyectos nuevos.   

In [1]:
import pandas as pd 
v = pd.read_csv('vivienda.csv')

In [ ]:
v.head()

# Sintesis de la información

In [2]:
v.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   longitud      20640 non-null  float64
 1   latitud       20640 non-null  float64
 2   antiguedad    20640 non-null  float64
 3   habitaciones  20640 non-null  float64
 4   dormitorios   20433 non-null  float64
 5   población     20640 non-null  float64
 6   hogares       20640 non-null  float64
 7   ingresos      20640 non-null  float64
 8   proximidad    20640 non-null  object 
 9   valor         20640 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [ ]:
v.proximidad.value_counts() 

In [ ]:
v.describe() 

# Visualizaciones 

In [ ]:
v.hist(figsize = (15,10)) 

In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 1, figsize = (10,6)) 

In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.1, figsize = (10,6)) 

In [ ]:
import matplotlib.pyplot as plt 

v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.1, figsize = (10,6),\
      s = v.población/100, label = 'Población', \
      c = 'valor', cmap = plt.get_cmap('jet'), colorbar = True)  

# Buscando correlaciones

Dado que el conjunto de datos no es demasiado grande, puede calcular fácilmente el coeficiente de correlación estándar (también llamado r de Pearson) entre cada par de atributos usando el método `corr()`:

In [ ]:
v.corr() 

Ahora veamos cómo cada atributo está correlacionado con la variable objetivo de valor promedio de vivienda por distrito: 

In [ ]:
v.corr().valor.sort_values(ascending = False) 

El coeficiente de correlación varía de –1 a 1.

Cuando es cercano a 1, significa que existe una fuerte correlación positiva; por ejemplo, el valor medio de la vivienda tiende a aumentar cuando aumenta el ingreso medio.

Cuando el coeficiente es cercano a –1, significa que existe una fuerte correlación negativa; puede ver una pequeña correlación negativa entre la latitud y el valor medio de la vivienda (es decir, los precios tienen una ligera tendencia a bajar cuando va hacia el norte).

Finalmente, los coeficientes cercanos a 0 significan que no existe una correlación lineal. 

La siguiente figura muestra varios gráficos junto con el coeficiente de correlación entre sus ejes horizontal y vertical.

<img src = 'https://github.com/marco-canas/intro-Machine-Learning/blob/main/classes/class_17_/coeficientes_de_correlaci%C3%B3n_est%C3%A1ndar.png?raw=true'>

In [ ]:
import matplotlib.pyplot as plt 

from pandas.plotting import scatter_matrix
attributes = ['valor', 'ingresos', 'habitaciones','antiguedad'] 
scatter_matrix(v[attributes], figsize=(12, 8))

plt.show() 

In [ ]:
v.plot(kind="scatter", x='ingresos', y='valor', alpha=0.1, figsize = (10,6))

plt.show() 

Esta gráfico revela algunas cosas.

* Primero, la correlación es de hecho muy fuerte; se puede ver claramente la tendencia al alza, y los puntos no están demasiado dispersos.

* En segundo lugar, el límite de precio que notamos anteriormente es claramente visible como una línea horizontal en $500,000.

Pero este gráfico revela otras líneas rectas menos obvias: 

* una línea horizontal alrededor de $450 000, 
* otra alrededor de $350 000, 
* quizás una alrededor de $280 000 y algunas más por debajo de esa. 

Es posible que desee intentar eliminar los distritos correspondientes para evitar que sus algoritmos aprendan a reproducir estas peculiaridades de los datos.

## Experimentando con combinaciones de atributos

Identificó algunas peculiaridades de los datos que quizás desee limpiar antes de enviar los datos a un algoritmo de aprendizaje automático y encontró correlaciones interesantes entre los atributos, en particular con el atributo objetivo.

Una última cosa que quizás desee hacer antes de preparar los datos para los algoritmos de Machine Learning es probar varias combinaciones de atributos.

Por ejemplo, el número total de habitaciones en un distrito no es muy útil si no sabe cuántos hogares hay.

Lo que realmente desea es el número de **habitaciones por hogar**. 

Del mismo modo, el número total de dormitorios por sí solo no es muy útil: probablemente quieras compararlo con el número de habitaciones.

Y la población por hogar también parece una combinación de atributos interesante para observar.

Vamos a crear estos nuevos atributos:

In [3]:
v['habitaciones_por_hogar'] = v.habitaciones/v.hogares 

In [4]:
v['dormitorios_por_habitación'] = v.dormitorios/v.habitaciones 

In [5]:
v['población_por_hogar'] = v.población/v.hogares 

Ahora veamos las correlaciones de nuevo: 

In [ ]:
v.corr().valor.sort_values(ascending = False) 

Note que:

* El nuevo atributo dormitorios_por_habitación está mucho más correlacionado con el valor medio de la casa que con el número total de habitaciones o dormitorios.

Aparentemente, las casas con una relación dormitorio/habitación más baja tienden a ser más caras.

El número de habitaciones por hogar también es más informativo que el número total de habitaciones en un distrito; obviamente, cuanto más grandes son las casas, más caras son.

Esta ronda de exploración no tiene que ser absolutamente minuciosa; el punto es comenzar con el pie derecho y obtener rápidamente información que lo ayudará a obtener un primer prototipo razonablemente bueno.

Pero este es un proceso iterativo: una vez que tiene un prototipo en funcionamiento, puede analizar su resultado para obtener más información y volver a este paso de exploración.

## Dividir en entrenamiento y testeo

In [6]:
from sklearn.model_selection import train_test_split

v_train, v_test = train_test_split(v, test_size = 0.2, random_state = 513) 



## Preparar los datos para los algoritmos de Machine Learning 

Es hora de preparar los datos para sus algoritmos de Machine Learning.

En lugar de hacer esto manualmente, debe escribir funciones para este propósito, por varias buenas razones:

Esto le permitirá reproducir estas transformaciones fácilmente en cualquier
conjunto de datos (por ejemplo, la próxima vez que obtenga un conjunto de datos nuevo).

Gradualmente construirá una biblioteca de funciones de transformación que puede reutilizar en futuros proyectos.

Puede usar estas funciones en su sistema en vivo para transformar el nuevo datos antes de alimentarlos a sus algoritmos.

Esto le permitirá probar fácilmente varias transformaciones y ver qué combinación de transformaciones funciona mejor.

Pero primero volvamos a un conjunto de entrenamiento limpio (copiando strat_train_set una vez más).

También separemos los predictores y las etiquetas, ya que no necesariamente queremos aplicar las mismas transformaciones a los predictores y los valores objetivo (tenga en cuenta que drop() crea una copia de los datos y no afecta a strat_train_set):

## Separar los datos en `X` y `y`

In [7]:
v = v_train.drop('valor', axis = 1)

In [8]:
v_labels = v_train.valor 

## Limpieza de datos

La mayoría de los algoritmos de aprendizaje automático no pueden funcionar con atributos faltantes, así que creemos algunas funciones para encargarnos de ellos. 

Vimos anteriormente que el atributo `dormitorios` tiene algunos valores faltantes, así que arreglemos esto.

Tienes tres opciones:

1. Deshacerse de los distritos correspondientes.
2. Deshágase de todo el atributo.
3. Establezca los valores en algún valor (cero, la media, la mediana, etc.).

Puede lograr esto fácilmente usando los métodos `dropna()`, `drop()` y `fillna()`:

### Probemos las tres opciones con copias de `v`

In [11]:
v1 = pd.concat((v.copy(),v_labels),axis = 1)  

In [12]:
v1.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 18223 to 17044
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitud                    16512 non-null  float64
 1   latitud                     16512 non-null  float64
 2   antiguedad                  16512 non-null  float64
 3   habitaciones                16512 non-null  float64
 4   dormitorios                 16348 non-null  float64
 5   población                   16512 non-null  float64
 6   hogares                     16512 non-null  float64
 7   ingresos                    16512 non-null  float64
 8   proximidad                  16512 non-null  object 
 9   habitaciones_por_hogar      16512 non-null  float64
 10  dormitorios_por_habitación  16348 non-null  float64
 11  población_por_hogar         16512 non-null  float64
 12  valor                       16512 non-null  float64
dtypes: float64(12), object(1)
m

In [14]:
len(v1.dormitorios)

16512

In [16]:
v1.dropna().info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16348 entries, 18223 to 17044
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitud                    16348 non-null  float64
 1   latitud                     16348 non-null  float64
 2   antiguedad                  16348 non-null  float64
 3   habitaciones                16348 non-null  float64
 4   dormitorios                 16348 non-null  float64
 5   población                   16348 non-null  float64
 6   hogares                     16348 non-null  float64
 7   ingresos                    16348 non-null  float64
 8   proximidad                  16348 non-null  object 
 9   habitaciones_por_hogar      16348 non-null  float64
 10  dormitorios_por_habitación  16348 non-null  float64
 11  población_por_hogar         16348 non-null  float64
 12  valor                       16348 non-null  float64
dtypes: float64(12), object(1)
m

In [17]:
v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 18223 to 17044
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitud                    16512 non-null  float64
 1   latitud                     16512 non-null  float64
 2   antiguedad                  16512 non-null  float64
 3   habitaciones                16512 non-null  float64
 4   dormitorios                 16348 non-null  float64
 5   población                   16512 non-null  float64
 6   hogares                     16512 non-null  float64
 7   ingresos                    16512 non-null  float64
 8   proximidad                  16512 non-null  object 
 9   habitaciones_por_hogar      16512 non-null  float64
 10  dormitorios_por_habitación  16348 non-null  float64
 11  población_por_hogar         16512 non-null  float64
 12  valor                       16512 non-null  float64
dtypes: float64(12), object(1)
m

## Segunda opción de tratamiento de datos faltantes

Para nuestro caso toca eliminar dos atributos 

In [19]:
v2 = v.copy() 

In [22]:
v2.drop(['dormitorios', 'dormitorios_por_habitación'], axis = 1).info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 18223 to 17044
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   longitud                16512 non-null  float64
 1   latitud                 16512 non-null  float64
 2   antiguedad              16512 non-null  float64
 3   habitaciones            16512 non-null  float64
 4   población               16512 non-null  float64
 5   hogares                 16512 non-null  float64
 6   ingresos                16512 non-null  float64
 7   proximidad              16512 non-null  object 
 8   habitaciones_por_hogar  16512 non-null  float64
 9   población_por_hogar     16512 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


### Tercera opción

In [23]:
v3 = v.copy() 

In [24]:
mediana = v3.dormitorios.median() 

In [25]:
v3.fillna(mediana) 

,longitud,latitud,antiguedad,habitaciones,dormitorios,población,hogares,ingresos,proximidad,habitaciones_por_hogar,dormitorios_por_habitación,población_por_hogar
18223,-122.08,37.41,20.0,1896.0,456.0,1069.0,436.0,4.6875,NEAR BAY,4.348624,0.240506,2.451835
6687,-118.07,34.14,42.0,3200.0,685.0,1668.0,628.0,3.3750,INLAND,5.095541,0.214062,2.656051
3854,-118.43,34.18,25.0,3830.0,1105.0,2328.0,1017.0,2.6238,<1H OCEAN,3.765978,0.288512,2.289086
11267,-117.97,33.80,35.0,2985.0,474.0,1614.0,453.0,5.4631,<1H OCEAN,6.589404,0.158794,3.562914
14498,-117.23,32.86,16.0,1675.0,354.0,604.0,332.0,5.2326,NEAR OCEAN,5.045181,0.211343,1.819277
...,...,...,...,...,...,...,...,...,...,...,...,...
12680,-121.39,38.55,25.0,2171.0,431.0,1053.0,422.0,3.5278,INLAND,5.144550,0.198526,2.495261
17219,-119.70,34.47,32.0,3725.0,569.0,1304.0,527.0,7.7261,<1H OCEAN,7.068311,0.152752,2.474383
18525,-122.04,36.97,45.0,1302.0,245.0,621.0,258.0,5.1806,NEAR OCEAN,5.046512,0.188172,2.406977
13822,-117.21,34.49,14.0,2125.0,348.0,1067.0,360.0,3.6333,INLAND,5.902778,0.163765,2.963889


In [26]:
v3.fillna(mediana).info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 18223 to 17044
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitud                    16512 non-null  float64
 1   latitud                     16512 non-null  float64
 2   antiguedad                  16512 non-null  float64
 3   habitaciones                16512 non-null  float64
 4   dormitorios                 16512 non-null  float64
 5   población                   16512 non-null  float64
 6   hogares                     16512 non-null  float64
 7   ingresos                    16512 non-null  float64
 8   proximidad                  16512 non-null  object 
 9   habitaciones_por_hogar      16512 non-null  float64
 10  dormitorios_por_habitación  16512 non-null  float64
 11  población_por_hogar         16512 non-null  float64
dtypes: float64(11), object(1)
memory usage: 1.6+ MB


Note que al llenar los datos faltantes en el atributo de `dormitorios`, automáticamente se actualiza el atributo de dormitorios por habitación. 

Si elige la opción 3, debe calcular el valor de la mediana en el conjunto de entrenamiento y usarlo para completar los valores que faltan en el conjunto de entrenamiento.

No olvide guardar el valor de la mediana que ha calculado.

Lo necesitará más adelante para reemplazar los valores faltantes en el conjunto de prueba cuando desee evaluar su sistema, y también una vez que el sistema entre en funcionamiento para reemplazar los valores faltantes en datos nuevos.

Scikit-Learn proporciona una clase útil para ocuparse de los valores faltantes:
`SimpleImputer`. Aquí está cómo usarlo.

Primero, debe crear una instancia de `SimpleImputer`, especificando que desea reemplazar los valores faltantes de cada atributo con la mediana de ese atributo:

In [30]:
from sklearn.impute import SimpleImputer 


In [31]:
imputer = SimpleImputer(strategy = 'median') 

Dado que la mediana solo se puede calcular en atributos numéricos, debe crear una copia de los datos sin el atributo de texto `proximidad`ocean_proximity:

In [32]:
v_num = v.drop(['proximidad'], axis = 1)

Ahora puede ajustar la instancia del imputer a los datos de entrenamiento usando el  método `fit()`:

In [33]:
imputer.fit(v_num)

SimpleImputer(strategy='median')

El imputador simplemente calculó la mediana de cada atributo y almacenó el resultado en su variable de instancia `statistics_`. 

Solo el atributo `dormitorios` tenía valores faltantes, pero no podemos estar seguros de que no habrá valores faltantes en los datos nuevos después de que el sistema entre en funcionamiento, por lo que es más seguro aplicar la imputación a todos los atributos numéricos:

In [34]:
imputer.statistics_

array([-1.18490000e+02,  3.42500000e+01,  2.90000000e+01,  2.11300000e+03,
        4.32000000e+02,  1.16200000e+03,  4.08000000e+02,  3.53850000e+00,
        5.22084367e+00,  2.03383550e-01,  2.82322522e+00])

In [35]:
v_num.median() 

longitud                      -118.490000
latitud                         34.250000
antiguedad                      29.000000
habitaciones                  2113.000000
dormitorios                    432.000000
población                     1162.000000
hogares                        408.000000
ingresos                         3.538500
habitaciones_por_hogar           5.220844
dormitorios_por_habitación       0.203384
población_por_hogar              2.823225
dtype: float64

Ahora puede usar esta computadora "entrenada" para transformar el conjunto de entrenamiento reemplazando los valores faltantes con las medianas aprendidas:

In [37]:
X = imputer.transform(v_num) 

El resultado es una matriz NumPy simple que contiene las características transformadas.

Si desea volver a colocarlo en un DataFrame de pandas, es simple:

In [38]:
v_num_tr = pd.DataFrame(X,columns = v_num.columns, index = v_num.index) 

In [39]:
v_num_tr.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 18223 to 17044
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitud                    16512 non-null  float64
 1   latitud                     16512 non-null  float64
 2   antiguedad                  16512 non-null  float64
 3   habitaciones                16512 non-null  float64
 4   dormitorios                 16512 non-null  float64
 5   población                   16512 non-null  float64
 6   hogares                     16512 non-null  float64
 7   ingresos                    16512 non-null  float64
 8   habitaciones_por_hogar      16512 non-null  float64
 9   dormitorios_por_habitación  16512 non-null  float64
 10  población_por_hogar         16512 non-null  float64
dtypes: float64(11)
memory usage: 1.5 MB


## Manipulación de datos categóricos

# Crear el DataFrame numérico y el DataFrame categórico

In [ ]:
v_num = v.drop('proximidad', axis = 1) 

In [ ]:
v_cat = v.proximidad 

1. Imputar los datos faltantes

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputar = SimpleImputer(strategy = 'median') 

In [ ]:
imputar.fit(v_num)

In [ ]:
X = imputar.transform(v_num) 

In [ ]:
v_num_imputado = pd.DataFrame(X, columns = v_num.columns, index = v_num.index) 

In [ ]:
v_num_imputado.info() 

In [ ]:
len(v_num_imputado) 

In [ ]:
v = v_num_imputado 

## 2. establecimiento de correlaciones. 

<img src = 'https://github.com/marco-canas/taca/blob/main/ref/geron/part_1/chap_2/4_visualize/figura_2_14_Standard_correlation_coefficient_of_various_datasets.PNG?raw=true'>

In [ ]:
v.corr() 

In [ ]:
v.corr().valor.sort_values(ascending =False)  

In [ ]:
v['dormitorios_por_habitaciones'] = v.dormitorios/v.habitaciones 

In [ ]:
v['población_por_hogares'] = v.población/v.hogares 

In [ ]:
v['habitaciones_por_hogar'] = v.habitaciones/v.hogares 

In [ ]:
v.corr().valor.sort_values(ascending = False) 

## 2. codificación de datos categóricos

In [ ]:
v_cat 

3. standarizar los datos